In [8]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.


In [9]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [11]:
import os, json, uuid, time
import pandas as pd
from tqdm import tqdm

In [12]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
gcp_key = user_secrets.get_secret("gcp_credentials")
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
for model in client.models.list():
    if "createTunedModel" in model.supported_actions:
        print(model.name)

In [ ]:
with open("/tmp/gcp_key.json", "w") as f:
    f.write(gcp_key)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/tmp/gcp_key.json"

# Set your GCP project and region
PROJECT_ID = "ai-detection-457406"
REGION = "us-central1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

from google.cloud import aiplatform
from google.cloud import storage

In [2]:
import pandas as pd

# Load your dataset
df = pd.read_csv("/kaggle/input/ai-vs-human-text/AI_Human.csv")

# Normalize column names
df.columns = [col.strip().lower() for col in df.columns]

# Convert label to readable format
df["generated"] = df["generated"].apply(lambda x: "ai" if x == 1.0 else "human")
df["text"] = df["text"].astype(str).str.strip()
df = df[["text", "generated"]]

# Estimate max samples per class (≈ 35%)
ai_count = df[df["generated"] == "ai"].shape[0]
human_count = df[df["generated"] == "human"].shape[0]
target_total = int(0.35 * (ai_count + human_count))
target_per_class = target_total // 2

print(f"Target per class: {target_per_class} samples")

# Sample balanced set
df_sampled = (
    df.groupby("generated")
    .apply(lambda x: x.sample(min(len(x), target_per_class), random_state=42))
    .reset_index(drop=True)
)

print(f"✅ Final sample size: {len(df_sampled)} rows")
df["generated"].value_counts()
df_sampled["generated"].value_counts()


Target per class: 85266 samples
✅ Final sample size: 170532 rows


/tmp/ipykernel_31/1022477164.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(min(len(x), target_per_class), random_state=42))


generated
ai       85266
human    85266
Name: count, dtype: int64

In [3]:
df_sampled["generated"].unique()

array(['ai', 'human'], dtype=object)

In [5]:
# Convert float to readable string labels
df_sampled["generated"] = df_sampled["generated"].apply(lambda x: "ai" if x == 1.0 else "human")

# Keep only necessary columns
df_sampled = df_sampled[["text", "generated"]]

# Optional: balance classes
min_count = df_sampled["generated"].value_counts().min()
df_sampled = df_sampled.groupby("generated").apply(lambda x: x.sample(min_count)).reset_index(drop=True)

df_sampled.head()

/tmp/ipykernel_31/3103721576.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_sampled.groupby("generated").apply(lambda x: x.sample(min_count)).reset_index(drop=True)


,text,generated
0,"Many people love libraries, and fa enjoy go sh...",human
1,"In recent years, there has been a growing move...",human
2,"Today, automobiles are responsible for 12 perc...",human
3,**Introduction**\n\nThe Face on Mass is a prom...,human
4,Video communication is a new technology that e...,human


In [ ]:
from collections.abc import Iterable
import random

import random

max_epochs = 1
max_steps = 250_000
max_examples = max_steps // max_epochs

# ✅ Convert df_sampled to formatted examples
examples = df_sampled.apply(
    lambda row: {
        "input_text": row["text"],
        "output_text": "This text was generated by AI." if row["generated"] == "ai" else "This text was written by a human."
    },
    axis=1
).tolist()


# ✅ Shuffle + trim if needed
random.shuffle(examples)

if len(examples) > max_examples:
    examples = examples[:max_examples]
    print(f"🔁 Downsampled to {len(examples)} examples to stay under {max_steps} steps")

# ✅ Update input_data
input_data = {"examples": examples}

# If you are re-running this lab, add your model_id here.
model_id = None

# Or try and find a recent tuning job.
if not model_id:
  queued_model = None
  # Newest models first.
  for m in reversed(client.tunings.list()):
    # Only look at newsgroup classification models.
    if m.name.startswith('tunedModels/newsgroup-classification-model'):
      # If there is a completed model, use the first (newest) one.
      if m.state.name == 'JOB_STATE_SUCCEEDED':
        model_id = m.name
        print('Found existing tuned model to reuse.')
        break

      elif m.state.name == 'JOB_STATE_RUNNING' and not queued_model:
        # If there's a model still queued, remember the most recent one.
        queued_model = m.name
  else:
    if queued_model:
      model_id = queued_model
      print('Found queued model, still waiting.')


# Upload the training data and queue the tuning job.
if not model_id:
    try:
        print("🚀 Launching tuning job (non-blocking)...")
        tuning_op = client.tunings.tune(
            base_model="models/gemini-1.5-flash-001-tuning",
            training_dataset=input_data,
            config=types.CreateTuningJobConfig(
                tuned_model_display_name="ai-detector-model",
                batch_size=4,
                epoch_count=1,
            ),
        )
        model_id = tuning_op.name
        print(f"✅ Job launched! Model ID: {model_id}")
    except Exception as e:
        print("❌ Failed to launch tuning job:")
        print(e)

print(model_id)

🚀 Launching tuning job (non-blocking)...


In [19]:
existing_jobs = client.tunings.list()

# Find most recent
for job in reversed(existing_jobs):
    if "ai-detector-model" in job.name:
        print(f"✅ Found job: {job.name} | Status: {job.state.name}")
        tuning_op = job
        break
else:
    print("❌ No recent tuning jobs found")

❌ No recent tuning jobs found
